In [1]:
import pandas as pd
import numpy as np
import urllib.request
import requests
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB 
from sklearn.linear_model import Perceptron
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re
from pymystem3 import Mystem
from stop_words import get_stop_words
from transformers import pipeline
import json
import os
from os.path import join

In [2]:
data_test_filename = join(os.environ["HOME"], "Datasets/llm_short.csv")
df_test = pd.read_csv(data_test_filename, sep=',', index_col=False)
df_test

,rating,text,req_id,text_converted
0,5.0,Приятное место. Обедали компанией в субботу. М...,norm:Приятное место. Были компанией в субботу....,Приятное место. Были компанией в субботу. Уютн...
1,4.0,"Хорошее кафе, есть для детей игровой уголок и ...","norm:Хорошее кафе, есть для детей игровой угол...","Хорошее кафе, есть для детей игровой уголок и ..."
2,4.0,\n4 с минусом. Туалет это дичь просто. Если ты...,norm:В туалете проблема: если мужчина встаёт к...,В туалете проблема: если мужчина встаёт к унит...
3,5.0,Прекрасное заведение и атмосфера. Все было оче...,norm:Прекрасное заведение и атмосфера. Все был...,Прекрасное заведение и атмосфера. Все было оче...
4,5.0,"Вежливый персонал, быстрая скорость отдачи зак...","norm:Вежливый персонал, быстрая скорость отдач...","Вежливый персонал, быстрая скорость отдачи зак..."
...,...,...,...,...
4995,3.0,Посмотрели на супер отзывы и пришли сюда за Ку...,norm:По отзывам решили заказать куриный шашлык...,По отзывам решили заказать куриный шашлык и са...
4996,5.0,Любимое место уже на протяжении многих лет ❤️ ...,norm:Любимое место уже на протяжении многих ле...,Любимое место уже на протяжении многих лет ❤️ ...
4997,5.0,Очень вкусное и свежее пиво по замечательной ц...,norm:Очень вкусное и свежее пиво по замечатель...,Очень вкусное и свежее пиво по замечательной ц...
4998,5.0,"Место не дешевое, но очень уютное. Прекрасный ...","norm:Место не дешёвое, но уютное. Прекрасный п...","Место не дешёвое, но уютное. Прекрасный персон..."


In [124]:
mask = df_test["text"].apply(len) > 500

# Use the mask to select rows where the condition is True
selected_rows = df_test[mask]
selected_rows

,rating,text,req_id,text_converted
7,1.0,качество персонал оставлять желать хороший при...,norm:Раньше я часто заглядывала в это заведени...,заглядывать заведение желание качество персона...
28,5.0,семья открывать новый заведение дом rest bar G...,norm:Недавно открыли для себя семейный rest&ba...,открывать семейный rest bar GAVAN располагать ...
111,3.0,впервые посещать заведение дзержинск область п...,norm:В вашем тексте можно выделить следующие о...,ваш текст выделять следующий основной идея впе...
147,4.0,постоянно делать покупка разный азбука качеств...,norm:Сотрудники в Азбуке обычно вежливые и кли...,сотрудник азбука вежливый клиентоориентированн...
188,1.0,избегать точка хинкали приносить холодный знат...,"norm:В кафе принесли хинкали холодными, мы сде...",кафе приносить хинкали холодный сделать заказ ...
...,...,...,...,...
4946,4.0,впечатление однозначный повезти обслуживание в...,norm:Впечатления разные: от отличного обслужив...,впечатление разный отличный обслуживание спешк...
4961,1.0,предпринимать 2 попытка посещать данный заведе...,norm:Во время отпуска мы дважды пытались посет...,отпуск дважды пытаться посещать заведение отка...
4968,5.0,уютный шикарный красивый оригинально поформлят...,"norm:Это уютное, красивое заведение в грузинск...",уютный красивый заведение грузинский стиль бла...
4976,1.0,делать заказ заранее 21 02 понимать 22 02 пред...,norm:**Вот сокращённый текст:**\n\nСделали зак...,сокращать текст сделать заказ заранее опоздани...


In [136]:
print(selected_rows.iloc[6]['text'])
print(selected_rows.iloc[6]['text_converted'])

вчера отмечать свадьба кухня огонь вкусно атмосфера бал минус звезда официант начало банкет типо шуршать начинать тупить причем жестко грамотный админстиатор следить работа официант заведение иркутск равный поверять практически заведение иркутск сравнивать официант отказывать предостовление чай пожилой человек какой рамка лезть официант наглый термос идти надевать некоторый словесный манипуляция говорить приносить взрослый делать одолжение мелочь смазывать шикарный вечер уровень надеяться руководитель данный заведение обращать внимание факт

вчера отмечать свадьба уровень кухня шикарный подводить официант начало банкет шуршать начинать жестко тупить официант отказывать чай пожилой человек замечание обещать обслуживать неохота надеяться руководитель заведение обращать внимание факт



In [141]:
quantiles = df_test['text_converted'].apply(len).quantile([0.1, 0.5, 0.9])

print("25th percentile (Q1):", quantiles[0.1])
print("50th percentile (median):", quantiles[0.5])
print("75th percentile (Q3):", quantiles[0.9])

25th percentile (Q1): 66.0
50th percentile (median): 122.5
75th percentile (Q3): 181.0


In [8]:
df_feltered = df_test[df_test['text_converted'].str.contains('[а-яА-Я]', na=False)]
df_test = df_feltered.copy()

In [9]:
data_train_filename = join(os.environ["HOME"], "Datasets/yandex_data_train.csv")
df_train = pd.read_csv(data_train_filename, sep=',', index_col=False, usecols=lambda column: column not in ['Unnamed: 0'])
df_train.head(500)

,rating,text
0,4.0,"Ресторан в центре Москвы, в замечательном мест..."
1,4.0,"Кухня не плохая, довольно таки вкусно. Но для ..."
2,5.0,"Отличный ресторан, попробовали очень много блю..."
3,4.0,"Бизнес-встречу здесь планировать провально , а..."
4,5.0,Завтракали в этом ресторане при гостинице. Все...
...,...,...
495,3.0,Еда была не вкусная. Тыквенный суп слишком кис...
496,5.0,Кафе прекрасное. Средний чек на двоих(2000 с ч...
497,5.0,"Просто волшебно, невероятно, бесподобно! \nСка..."
498,5.0,В данном ресторане люблю бывать. Нравится все:...


In [10]:
stop_words = get_stop_words('russian')
def remove_russian_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

mystem = Mystem()
# Function to apply stemming to Russian text
def stem_russian_text(text):
    tokens = mystem.lemmatize(text)
    return ''.join(tokens)


def prepare(data, col='text'):
    # Step 1: Remove punctuation
    data[col] = data[col].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

    # Step 2: Remove stop words
    data[col] = data[col].apply(remove_russian_stop_words)

    # Step 3: Stem words
    data[col] = data[col].apply(stem_russian_text)


In [11]:
prepare(df_test)
prepare(df_test, 'text_converted')
prepare(df_train)

In [13]:
df_test["text"]

0       приятный место обедать компания суббота место ...
1       хороший кафе ребенок игровой уголок детский ме...
2       n4 минус туалет дичь мужчина вставать унитаз п...
3       прекрасный заведение атмосфера вкусно божестве...
4       вежливый персонал быстрый скорость отдача зака...
                              ...                        
4995    посмотреть супер отзыв приходить сюда куриный ...
4996    любимый место протяжение многий подводить обсл...
4997    вкусный свежий пиво замечательный цена пкухня ...
4998    место дешевый уютный прекрасный персонал кухня...
4999    посещать заведение настолько понравиться решат...
Name: text, Length: 5000, dtype: object

In [ ]:
df_test["text_ant"] = df_test['text'].apply(swap_adjectives)

In [14]:
df_all = pd.concat([df_train['text'], df_test['text'], df_test['text_converted']])
df_all

0       ресторан центр москва замечательный место хоро...
1       кухня плохой таки вкусно обстановка подвал дор...
2       отличный ресторан попробовать блюдо вкусно пер...
3       бизнес встреча планировать провальный проводит...
4       завтракать ресторан гостиница невероятно вкусн...
                              ...                        
4995    отзыв решать заказывать куриный шашлык салат к...
4996    любимый место протяжение многий подводить обсл...
4997    вкусный свежий пиво замечательный цена кухня в...
4998    место дешевый уютный прекрасный персонал вкусн...
4999    заведение понравиться вкусно огромный порция н...
Length: 74537, dtype: object

In [15]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df_all)

TfidfVectorizer()

In [113]:
X_train = vectorizer.transform(df_train['text'])
y_train = df_train['rating'].apply(lambda x: x > 4)

X_test = vectorizer.transform(df_test['text'])
X_drift_tesst = vectorizer.transform(df_test['text_converted'])
y_test = df_test['rating'].apply(lambda x: x > 4)

In [114]:
clf = MultinomialNB(alpha=0.2)
clf.fit(X_train, y_train)

MultinomialNB(alpha=0.2)

In [115]:
# NO DRIFT
y_pred = clf.predict(X_test)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred, target_names=["Not 5 star", "5 star"]))

Accuracy: 0.8842
              precision    recall  f1-score   support

  Not 5 star       0.90      0.58      0.71      1200
      5 star       0.88      0.98      0.93      3800

    accuracy                           0.88      5000
   macro avg       0.89      0.78      0.82      5000
weighted avg       0.89      0.88      0.87      5000



In [116]:
#  DRIFT
y_pred = clf.predict(X_drift_tesst)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred, target_names=["Not 5 star", "5 star"]))

Accuracy: 0.8644
              precision    recall  f1-score   support

  Not 5 star       0.94      0.47      0.62      1200
      5 star       0.85      0.99      0.92      3800

    accuracy                           0.86      5000
   macro avg       0.89      0.73      0.77      5000
weighted avg       0.87      0.86      0.85      5000



In [117]:
X_test_retrain, X_test_30, y_test_retrain, y_test_30 = train_test_split(X_drift_tesst, y_test, test_size=0.3, random_state=42)

In [118]:
clf.partial_fit(X_test_retrain, y_test_retrain)

MultinomialNB(alpha=0.2)

In [119]:
y_pred = clf.predict(X_test_30)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test_30, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test_30, y_pred, target_names=["Not 5 star", "5 star"]))

Accuracy: 0.878
              precision    recall  f1-score   support

  Not 5 star       0.92      0.52      0.66       348
      5 star       0.87      0.99      0.93      1152

    accuracy                           0.88      1500
   macro avg       0.90      0.75      0.79      1500
weighted avg       0.88      0.88      0.86      1500



In [120]:
# NO DRIFT
y_pred = clf.predict(X_test)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred, target_names=["Not 5 star", "5 star"]))

Accuracy: 0.9062
              precision    recall  f1-score   support

  Not 5 star       0.92      0.66      0.77      1200
      5 star       0.90      0.98      0.94      3800

    accuracy                           0.91      5000
   macro avg       0.91      0.82      0.86      5000
weighted avg       0.91      0.91      0.90      5000



In [142]:
period1_text = df_train['text']
period2_text = df_test['text']

p = vectorizer.transform(period1_text)
q = vectorizer.transform(period2_text)

# Convert to dense array for Hellinger distance calculation
p = p.toarray().mean(axis=0)
q = q.toarray().mean(axis=0)
np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q))**2)) / np.sqrt(2)

0.5728830336540907

In [122]:
import pandas as pd
from collections import Counter
import numpy as np
from scipy.stats import ks_2samp

def calculate_word_frequencies(text_data):
    # Tokenize text and count word frequencies
    tokenized_text = [word for sentence in text_data for word in sentence.split()]
    word_freq = Counter(tokenized_text)
    return word_freq

def detect_data_drift(part1_data, part2_data):
    # Calculate word frequencies for each part
    part1_word_freq = calculate_word_frequencies(part1_data)
    part2_word_freq = calculate_word_frequencies(part2_data)

    # Extract word frequencies and sort by word
    part1_word_freq_values = np.array(list(part1_word_freq.values()))
    part2_word_freq_values = np.array(list(part2_word_freq.values()))

    # Calculate Kolmogorov-Smirnov statistic
    ks_statistic, p_value = ks_2samp(part1_word_freq_values, part2_word_freq_values)

    # Define a threshold for significant drift (you can adjust this threshold)
    threshold = 0.05
    return p_value

    # Check for data drift
    if p_value < threshold:
        print("Data drift detected!")
    else:
        print("No significant data drift detected.")

# Sample data (replace with your dataset)
part1_data = list(df_train['text'])
part2_data = list(df_test['text'])

# Detect data drift
detect_data_drift(part1_data, part2_data)

5.21409626826448e-13